# consumer

In [12]:
# requirements #
import numpy as np 
import pandas as pd 
import asyncio
from azure.eventhub import EventData, EventHubConsumerClient, PartitionContext
from azure.storage import filedatalake
######
import time 
from io import BytesIO as bio 
import pickle 
import json 
import logging
logger = logging.getLogger("azure.eventhub")

In [13]:
ehub_conn_str = "Endpoint=sb://eventhubs-ns-1.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=Q2mbJQ5jgawhS5k+uVNFEOUgI2AhpujuQkv+zRT8gNg="
ehub_name = 'ehub1'
consumer_group = 'consumergrp1'
##########
datalake_conn_str = 'DefaultEndpointsProtocol=https;AccountName=ehubdatalake;AccountKey=0WKR2WkRMhZVpBdHlzzqC+zmXeECfoAo+26OBUynCUrK2++vmeHPfN2O4DQsV2RJBgUbwYDES7S1ktPcK1G4tA==;EndpointSuffix=core.windows.net'
dlake = filedatalake.DataLakeServiceClient.from_connection_string(
    conn_str=datalake_conn_str
)
file_system = 'consumerdata'

In [14]:
# dlake.create_file_system('consumerdata')
file = dlake.get_file_client(file_system, 'events/test1')

In [87]:
# with json
file.upload_data(json.dumps(dict(a='asdf', b=3492)), overwrite=True)
json.loads(bio(file.download_file().readall()).read())

{'a': 'asdf', 'b': 3492}

In [75]:
# with pickle
file.upload_data(pickle.dumps(dict(a='asdf', b=3492)), overwrite=True)
pickle.loads(bio(file.download_file().readall()).read())

{'a': 'asdf', 'b': 3492}

In [73]:
json.loads(bio(file.download_file().readall()).read())

{'a': 'asdf', 'b': 3492}

# catch events and upload to dlake

# binary events

In [17]:
consumer = EventHubConsumerClient.from_connection_string(
    conn_str=ehub_conn_str, 
    consumer_group=consumer_group, 
    eventhub_name=ehub_name,
    logging_enable=True
)

eventobj = None 
start = time.time()

def on_event(partition_context: PartitionContext, event: EventData):
    # local checks
    global eventobj
    eventobj = event 
    for record in event.body: 
        print(json.loads(record))
        # datalake storage
        file = dlake.get_file_client(file_system, f'events/{event.sequence_number}')
        file.upload_data(record, overwrite=True)
        # If the operation is i/o intensive, multi-thread will have better performance.
    logger.info("Received event from partition: {}".format(partition_context.partition_id))
    secs = (time.time() - start)
    if secs > 30:
        consumer.close()


with consumer:
    consumer.receive(
        on_event=on_event,
#         partition_id=0
        prefetch=300, 
        starting_position="@latest",   # "@latest" for receiving only new events.
#         starting_position=-1,  
    )

[[5.204201902816778, 133.57117333313684, 158.28522559290937], [4.767382921362228, 142.26051573765682, 135.08494066686697], [6.5085422435454605, 127.11965760330047, 149.67587632314556], [4.731306965513948, 124.03762754778153, 153.3464256287288], [5.467131088011919, 140.96279735494738, 149.28641191957936]]
[[5.1250230201507865, 118.66823648036699, 130.63068658072234], [5.804795671633542, 138.41868057957055, 143.1079514455482], [6.446577957712487, 127.36414819887352, 137.49581707441058], [6.1472234635229, 136.62957320445136, 128.6657020510627], [5.829338048010589, 127.24029693355733, 145.35884243166697]]


# string events

In [97]:
consumer = EventHubConsumerClient.from_connection_string(
    conn_str=ehub_conn_str, 
    consumer_group=consumer_group, 
    eventhub_name=ehub_name,
    logging_enable=True
)

eventobj = None 
start = time.time()

def on_event(partition_context: PartitionContext, event: EventData):
    # local checks
    global eventobj
    eventobj = event 
    print(event.body_as_str())
    # datalake storage
    file = dlake.get_file_client(file_system, f'events/{event.sequence_number}')
    file.upload_data(event.body_as_str(), overwrite=True)
    # If the operation is i/o intensive, multi-thread will have better performance.
    logger.info("Received event from partition: {}".format(partition_context.partition_id))
    mins = (time.time() - start) / 60
    if mins > 1:
        consumer.close()


with consumer:
    consumer.receive(
        on_event=on_event,
#         partition_id=0
        prefetch=300,
        starting_position="@latest",   # "@latest" for receiving only new events.
#         starting_position=-1,  
    )

nb Event 1
nb Event 2
nb Event 3
nb Event 1


In [88]:
eventobj 

EventData(body='nb Event 1', properties=None, offset=2157048, sequence_number=26996, partition_key=None, enqueued_time=datetime.datetime(2021, 1, 3, 5, 32, 6, 419000, tzinfo=datetime.timezone.utc))

In [89]:
eventobj.body_as_str()

'nb Event 1'

In [90]:
eventobj.sequence_number

26996

In [96]:
file = dlake.get_file_client(file_system, f'testlog/{eventobj.sequence_number}')
file.upload_data(eventobj.body_as_str(), overwrite=True)

{'date': datetime.datetime(2021, 1, 3, 5, 36, 47, tzinfo=datetime.timezone.utc),
 'etag': '"0x8D8AFA990CB6EA5"',
 'last_modified': datetime.datetime(2021, 1, 3, 5, 36, 48, tzinfo=datetime.timezone.utc),
 'content_length': 0,
 'client_request_id': 'ac7a4c22-4d85-11eb-89ac-acde48001122',
 'request_id': 'b3fd4c4a-a01f-0016-3392-e11007000000',
 'version': '2020-02-10'}

In [95]:
file.upload_data(b'asdf', overwrite=True)

{'date': datetime.datetime(2021, 1, 3, 5, 36, 35, tzinfo=datetime.timezone.utc),
 'etag': '"0x8D8AFA9895A224E"',
 'last_modified': datetime.datetime(2021, 1, 3, 5, 36, 36, tzinfo=datetime.timezone.utc),
 'content_length': 0,
 'client_request_id': 'a5076eac-4d85-11eb-89ac-acde48001122',
 'request_id': 'b3fd4c3d-a01f-0016-2c92-e11007000000',
 'version': '2020-02-10'}